# Introduction

The `Interfaces.jl` package makes it easy to create a struct that holds a fixed set of concretely-typed function wrappers. You can use this to, for example, efficiently call a function on a collection of objects of different types without the performance penalty of dynamic dispatch. 

In [1]:
using BenchmarkTools
using Base.Test

# Example

This is easiest to show with an example. Let's create two concrete types `B` and `C` which both inherit from an abstract type `A` (having an abstract parent isn't necessary for Interfaces.jl to work, but it can be nice). 

We'll define a function `foo(::A, y::Any)` that acts on any `A` and just adds its `x` field to `y`:

In [2]:
abstract type A end

struct B <: A
    x::Int
end

struct C <: A
    x::Float64
end

foo(a::A, y) = a.x + y

foo (generic function with 1 method)

We can now create a `B` and a `C` and call the `foo` function on them:

In [3]:
b = B(1)
c = C(2.0)
@test foo(b, 2) === 3
@test foo(c, 2) === 4.0

Test Passed

But because `B` and `C` are different types, if we try to call `foo` on a vector of `B`s and `C`s, it will be slow:

In [4]:
y = zeros(2)
bc = [b, c]
@benchmark $y .= foo.($bc, 2.0)

BenchmarkTools.Trial: 
  memory estimate:  80 bytes
  allocs estimate:  5
  --------------
  minimum time:     438.717 ns (0.00% GC)
  median time:      496.343 ns (0.00% GC)
  mean time:        557.289 ns (0.50% GC)
  maximum time:     5.196 μs (86.87% GC)
  --------------
  samples:          10000
  evals/sample:     198

Instead, we can define an `interface` which wraps up the `foo` method for a particular set of input and output types:

In [5]:
using ConcreteInterfaces

In [6]:
@interface FooInterface(self::A) begin
    foo(y::Float64)::Float64 = foo(self, y)
end

foo (generic function with 2 methods)

We can create a `FooInterface` from any `::A`, and those interfaces will have the *same* type regardless of the specific type of their input:

In [7]:
i1 = FooInterface(b)
i2 = FooInterface(c)
@test typeof(i1) === typeof(i2)

Test Passed

The `@interface` macro automatically defines the `foo` function for our interfaces:

In [8]:
@test foo(i1, 2.0) === 3.0

# Type inference works just fine for our interfaces, too:
@inferred(foo(i1, 2.0))
@inferred(i1.foo(2.0))

3.0

And now we can store a vector of `FooInteface`s and efficiently call the `foo` method on each of them:

In [9]:
I = [i1, i2]
@test eltype(I) === typeof(i1) === typeof(i2)
y = zeros(2)
@benchmark $y .= foo.($I, 2.0)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     17.792 ns (0.00% GC)
  median time:      21.870 ns (0.00% GC)
  mean time:        24.841 ns (0.00% GC)
  maximum time:     93.498 ns (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     995

### Retrieving Arguments

The `FooInterface` type stores the function pointers used in its interface as well as the arguments which were used to construct it:

In [10]:
@test FooInterface(b).self === b

Test Passed

And the function pointers themselves can also be directly accessed:

In [11]:
i1 = FooInterface(b)
@test i1.foo(2.0) === foo(i1, 2.0) === 3.0

Test Passed

## More Complex Interfaces

An interface can have multiple methods:

In [12]:
@interface FooAndXInterface(self::A) begin
    foo(y::Float64)::Float64 = foo(self, y)
    getx()::Float64 = convert(Float64, self.x)
end

i3 = FooAndXInterface(b)
@test getx(i3) === convert(Float64, b.x)

Test Passed

### Parametric Interfaces

And an interface can also have type parameters:

In [13]:
@interface ParametricFooInterface{T}(self::A) begin
    foo(y::T)::T = foo(self, y)
end

i4 = ParametricFooInterface{Complex{Float64}}(b)
@test @inferred(foo(i4, 1.0 + 0.5im)) === 2.0 + 0.5im

Test Passed

### Computed Interfaces

The interface's return types can even be computed functions of the input types:

In [14]:
@interface ComputedFooInterface{T}(self::A) begin
    foo(y::T)::Base.promote_op(+, T, Float64) = foo(self, y)
end

i5 = ComputedFooInterface{Int}(b)
@test @inferred(foo(i5, 2)) === 3.0

Test Passed